In [2]:
%matplotlib inline

In [3]:
import sympy
import math
import cmath
import numpy as np
import numpy.polynomial.polynomial as p
import matplotlib.pyplot as plt
from turtle import *
import re
from sympy.ntheory import discrete_log
from matplotlib.transforms import Affine2D
import skimage.io
import time

# Principal Component Analysis

Task:

4. Principal Component Analysis

Sometimes a projection of a higher-dimensional to a lower-dimensional space is useful. It's extremely useful if we want to get some visual understanding of a, say, 15D space, in 3D or even 2D. One algorithm which allows us to project multidimensional data into fewer dimensions while keeping the most important shapes and structures is called principal component analysis (PCA). You can explore this using the following checklist:

- What are eigenvalues and eigenvectors?
- What is the eigenbasis? What is the spectrum of a matrix?
- How do we compute the eigenvalues and eigenvectors of a matrix?
- What is projection?
- How does projection conserve some shapes? Think about an object casting a shadow
- How is the projection problem related to eigenvalues and eigenvectors?
- What is PCA?
- What are principal components? How many components are there (as a function of dimensions of the original space)?
- What is variance? What is explained variance?
- How do principal components relate to explained variance?
- How is PCA implemented? Implement and show
- Show some applications of PCA, e.g. reducing a 3D image to its first 2 principal components, plotting the 3D and 2D images
- Show a practical use of PCA, for example, trying to see features in a 15D space, projected in 3D.

## What is PCA?



## References:

<https://www.cs.cmu.edu/~tom/10701_sp11/slides/pca_schlens.pdf>

<https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf>

<https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c>

<https://towardsdatascience.com/the-mathematics-behind-principal-component-analysis-fff2d7f4b643>

<https://en.wikipedia.org/wiki/Principal_component_analysis>

<https://builtin.com/data-science/step-step-explanation-principal-component-analysis>

<https://heartbeat.fritz.ai/understanding-the-mathematics-behind-principal-component-analysis-efd7c9ff0bb3>

<https://medium.com/analytics-vidhya/mathematics-behind-principal-component-analysis-pca-1cdff0a808a9>

<https://www.nature.com/articles/nmeth.4346>

<https://en.wikipedia.org/wiki/Norm_(mathematics)>

<https://en.wikipedia.org/wiki/Euclidean_distance>